In [1]:
from tensorflow import keras

from tensorflow.keras import models 
from tensorflow.keras import layers 
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 

In [2]:
pd.set_option('mode.chained_assignment', None)

# Preparing the dataset and creating the features and target arrays


n_sims = 10000
n_list = [i for i in range(n_sims)]


h0s = []
results = []


start = 0
end = 10000

x = []
y =[]

for i in n_list[start:end]:
	
	data = np.load(f'input/data_{i+1}.npy')
	x.append(data[:, :2])
	y.append(data[:, 2:][0][0])
	print(f'Status: {((i+1)/end * 100):.2f} %', end='\r')


x = np.array(x)
# x[:,:,0] /= x[:,:,0].max()

y = np.array(y)



kfold = KFold(n_splits=5, shuffle=True)


In [3]:
# K-fold Cross Validation

fold_no = 1

mae_per_fold = []
mse_per_fold = []
r2_per_fold = []
rmse_per_fold = []


for train, test in kfold.split(x, y):

    model = models.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(16))
    model.add(layers.Dense(1))
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer='adam', metrics=keras.metrics.MeanAbsoluteError())


    print('\n------------------------------------------------------------------------\n')
    print(f'\nTraining for fold {fold_no} ...\n')


    history = model.fit(x[train], y[train], epochs=25, batch_size=10, verbose=0)


    # scores = model.evaluate(x[test], y[test], verbose=0)

    y_pred = model.predict(x[test])


    r2 = r2_score(y[test], y_pred)
    mse = mean_squared_error(y[test], y_pred)
    mae = mean_absolute_error(y[test], y_pred)
    rmse = root_mean_squared_error(y[test], y_pred)


    print(f'Score for fold {fold_no}: \nMAE = {mae:.3f}\nR2 = {r2:.4f}\nMSE = {mse:.3f}\nRMSE = {rmse:.3f}')



    mae_per_fold.append(mae)
    mse_per_fold.append(mse)
    r2_per_fold.append(r2)
    rmse_per_fold.append(rmse)

    # Increase fold number
    fold_no += 1


------------------------------------------------------------------------


Training for fold 1 ...

63/63 [==============================] - 0s 1ms/step
Score for fold 1: MAE = 0.101
R2 = 0.9994
MSE = 0.010
RMSE = 0.101

------------------------------------------------------------------------


Training for fold 2 ...

63/63 [==============================] - 0s 1ms/step
Score for fold 2: MAE = 0.178
R2 = 0.9982
MSE = 0.034
RMSE = 0.183

------------------------------------------------------------------------


Training for fold 3 ...

63/63 [==============================] - 0s 2ms/step
Score for fold 3: MAE = 0.065
R2 = 0.9997
MSE = 0.005
RMSE = 0.073

------------------------------------------------------------------------


Training for fold 4 ...

63/63 [==============================] - 0s 2ms/step
Score for fold 4: MAE = 0.094
R2 = 0.9993
MSE = 0.013
RMSE = 0.116

------------------------------------------------------------------------


Training for fold 5 ...

63/63 [========

In [4]:
np.mean(mae_per_fold)


0.09363305512926978

In [5]:
np.mean(mse_per_fold)


0.012807394747141399

In [6]:
np.mean(r2_per_fold)


0.9993181997615675

In [7]:
np.mean(rmse_per_fold)

0.10225208229042097